In [ ]:
import PublicDataReader as pdr
from config import OpenAPI
import pandas as pd

# DataFrame 디스플레이 설정
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)

print(pdr.__all__)
print(pdr.__version__)
# print(pdr.__info__)

In [ ]:
serviceKey = OpenAPI['molit']

In [ ]:
molit = pdr.Transaction(serviceKey, debug=True)

In [ ]:
prod = "아파트"
trans = "전월세"

df = molit.read_data(prod, trans, "41135", "202110")

## Semas Class

In [ ]:
serviceKey = OpenAPI['molit']
semas = pdr.StoreInfo(serviceKey, debug=True)

## Transaction Class

In [ ]:
# 법정동코드 목록
code = pdr.code_list()
code.head()

In [ ]:
# 공공 데이터 포털 Open API 서비스키
serviceKey = OpenAPI['molit']
self = pdr.Transaction(serviceKey, debug=True)

prod="아파트"
trans="매매"
sigunguCode="41135"
yearMonth="202101"

df = self.read_data(prod, trans, sigunguCode, yearMonth)
df2 = self.collect_data(prod, trans, sigunguCode, "202101", "202103")

## Building Class

In [ ]:
# 공공 데이터 포털 Open API 서비스키
serviceKey = OpenAPI['molit']
self = pdr.Building(serviceKey, debug=True)

In [ ]:
category = "오수정화시설"

sigunguCd = "41135"
bjdongCd = "11000"
bun = "0542"
ji = "0000"

df = self.read_data(category, sigunguCd=sigunguCd, bjdongCd=bjdongCd, bun=bun, ji=ji)
df

In [ ]:
sigunguCd = "41135"
bjdongCd = "11000"
bun = "0542"
ji = "0000"
for category in self.metaDict.keys():
    print(f">>>>>> {category}")
    df = self.read_data(category, sigunguCd=sigunguCd, bjdongCd=bjdongCd, bun=bun, ji=ji)
    print(df.head(1))

## 컬럼 리스트 추출

In [ ]:
# LAWD_CD=11680
# DEAL_YMD=202110


# for prod in ['아파트','오피스텔','단독다가구','연립다세대','상업업무용','분양입주권','토지']:
#     for trans in ['매매', '전월세']:
#         try:
#             print()
#             print(f">> {prod} {trans}")
#             print()
#             service_url = self.metaDict[prod][trans]['url']

#             # URL
#             url=f"""\
# {service_url}\
# &LAWD_CD={str(LAWD_CD)}\
# &DEAL_YMD={str(DEAL_YMD)}\
# &numOfRows=99999\
#             """

#             result = requests.get(url, verify=False)
#             xmlsoup = BeautifulSoup(result.text, "lxml-xml")
#             item = xmlsoup.find("item")

#             col_list = []
#             for i in item:
#                 col_list.append(i.name)
#             for col in col_list: 
#                 print(f"'{col}'", end=',')
                
#         except:
#             print("Error")
#             pass

### Features

- 공통

    - 로깅 기능
        - 세션 초기화 시 `debug` 인자 추가
            - False(Default): 오류 레벨만 출력
            - True: 모든 레벨 출력
        - OPEN API 에러 코드 출력

    - Air Korea Open API 모듈 제거
    
    - 법정동코드 테이블 조회 함수 추가

- 부동산 실거래가

    - 데이터 조회 메서드 일원화
        - molit.read_data('아파트', '매매', '11680', '202111')
            - 서비스별 End-point 및 컬럼 매핑 딕셔너리 추가
    - 데이터 수집 메서드 신규 메서드 추가
        - molit.collect_data('아파트', '매매', '11680', '202101', '202103')

- 건축물대장
    
    - 데이터 조회 메서드 일원화
        - molit.read_data(category, sigunguCd=sigunguCd, bjdongCd=bjdongCd, bun=bun, ji=ji)